## Install packages

In [1]:
!pip3 install --user pandas
!pip3 install --user numpy
!pip3 install --user joblib
!pip3 install --user scipy

## Import librairies and functions

In [2]:
import joblib
import os
import pandas as pd
import numpy as np
from scipy import interpolate
import pandas as pd
from typing import Dict, List


def load(directory: str, filename: str) -> pd.DataFrame:
    """
    Load a dataframe from pickle format.
    """
    path = os.path.join(directory, filename)
    df = joblib.load(path)
    return df


def save(df: pd.DataFrame, directory: str, filename: str) -> pd.DataFrame:
    """
    Save a dataframe into pickle format.
    """
    path = os.path.join(directory, filename)
    joblib.dump(df, path)

    
def clean_batch_of_raw_data_time_windowed(
    df: pd.DataFrame,
    time_window: float,
    means,
    stds
) -> pd.DataFrame:
    """
    Clean a batch of raw data.

    Parameters
    ----------
    df : pd.DataFrame
        A batch of raw messages.

    Returns
    -------
    pd.DataFrame
        Batch of clean data
    """
    df = convert_objects_to_float(df)
    df = add_NSG_columns_and_sort_columns_alphabetically(df)
    df = angle_transform(df)
    df = remove_disturbance(df)
    df = keep_fixed_time_data(df, time_window)
    df = interpolation_reconstruction_delta_T(df, 0.01)
    df = normalise_df(df, means, stds)
    df = df.droplevel(1, axis=1)
    return df


def load_clean_save_raw_data_by_batch(
    root_dir: str,
    output_data_dir: str,
    sep: str,
    header: List[int],
    time_window: float,
    means,
    stds
) -> pd.DataFrame:
    """
    Load, clean and save raw data batch by batch.

    Parameters
    ----------
    root_dir : str
        Directory to probe for loading.
    output_data_dir : str
        Directory where to save cleaned data.
    sep : str
        Separator used when reading CSVs.
    cols_to_keep : List[str]
        List of columns to keep. All other columns are dropped.
    """
    for i, file in enumerate(os.listdir(root_dir)):
        if file.endswith('results_cascs.csv'):
            continue
        else:
            print(i, file)
            file_path = os.path.join(root_dir, file)
            df = pd.read_csv(file_path, sep=',', header=[0,1])
            try:
                df = clean_batch_of_raw_data_time_windowed(df, time_window, means, stds)
                save(df, output_data_dir, file.replace('.csv', '.pkl'))
            except:
                print("----- Error in processing ", file, " --- Mostly due to cascading failure ")
            

def get_list_simulations(root_dir: str) -> List[str]:
    """
    Get a list with all the name of the simulation csv files.
    """
    file_list_simulations = []
    for i, file in enumerate(os.listdir(root_dir)):
        if file.endswith('results_cascs.csv'):
            continue
        else:
            file_list_simulations.append(file)
    return file_list_simulations


def get_dict_scenario_csv(file_list_simulations: List[str]) -> Dict[int, str]:
    """
    Build dictionary of scenario number and the corresponding name of the csv file
    with simulation data.
    """
    simulation_files_dict = {}
    for file_name in file_list_simulations:
        simulation_files_dict[int(file_name.split("_")[0])] = file_name
    return simulation_files_dict


def add_NSG_columns_and_sort_columns_alphabetically(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Add NSG columns when missing from dataframe in order
    to get homogeneous set of columns in each dataframe.
    """
    cols_to_add = [('NSG_1', 'Active Power in MW'), ('NSG_1', 'Reactive Power in Mvar' ), ('NSG_2', 'Active Power in MW'), ('NSG_3', 'Active Power in MW')]
    for col in cols_to_add:
        if not col in df.columns:
            df[col] = '0'
    df = df.sort_index(axis=1)
    return df


def convert_objects_to_float(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert columns with object dtype to floats in order to use them in models.
    """
    indicator = df.dtypes == 'object'
    categorical_columns = df.columns[indicator].tolist()
    for col in categorical_columns:
        df[col] = df[col].astype('float')
    return df


def interpolation_reconstruction_delta_T(df: pd.DataFrame, delta_T: float = 0.01) -> pd.DataFrame:
    """
    Compute interpolated timestamp for each scenario.
    The interpolation is made on a regular grid of x-values spanning the same interval as during the simulatio with a delta_T specified by user.
    IMPORTANT - ASSUMES time.min() < 1.08
    
    Parameters
    ----------
    df : pd.DataFrame
        Simulation dataframe
    delta_T : float
        The time difference between consecutive data.
    """
    list_concat = []
    for col in df.columns[1:]:
        X = df['All calculations', 'Time in s'].values
        y = df[col].values
        f = interpolate.interp1d(X, y, kind='slinear')
        xgrid = np.arange(1.08, X.max(), delta_T)
        ygrid = f(xgrid)
        to_concat = pd.DataFrame(
            {('All calculations', 'Time in s'): xgrid, col: ygrid})
        list_concat.append(to_concat)
    dfs = [df.set_index(('All calculations', 'Time in s'))
           for df in list_concat]
    interpolate_construction = pd.concat(dfs, axis=1)
    return interpolate_construction.reset_index()


def pre_normalisation_mean_and_std(df):
    """
    find mean and std of a given df for
    normalisation purposes

    """
    x = df.values.astype(float)
    mean = x.mean(axis=0)
    std = x.std(axis=0)
    return mean, std


def find_means_and_stds(sample_size, root_dir, simulation_files_dict):
    """
    find the average mean and std of each column from
    a sample of size sample_size
    run this before running clean_batch_of_raw_data()

    """
    means = []
    stds = []
    for i in range(sample_size):
        df = pd.read_csv(os.path.join(root_dir,
                                    simulation_files_dict[i+1]), sep=',', header=[0, 1])
        df = convert_objects_to_float(df)
        df = add_NSG_columns_and_sort_columns_alphabetically(df)
        df = interpolation_reconstruction_delta_T(df, delta_T=0.01)
        mean, std = pre_normalisation_mean_and_std(df)
        means.append(mean)
        stds.append(std)
    return np.concatenate((means)).reshape(sample_size, 250).mean(axis=0).reshape(1, 250), np.concatenate((stds)).reshape(sample_size, 250).mean(axis=0).reshape(1, 250)


def normalise_df(df, means, stds):
    """
    normalise a single dataframe column by column
    it is probably possible to vectorise this and make it faster...
    means and stds can either be saved to the workspace or can
    change clean_batch_of_raw_data() to take them as inputs

    """
    for i in range(1, 250):
        df.iloc[:, i] = df.iloc[:, i].astype(float) - means[0][i]
        if stds[0][i] == 0:  # some variables may return zero st. dev. for certain batches
            continue
        else:
            df.iloc[:, i] = df.iloc[:, i].astype(float)/stds[0][i]
    return df


def remove_disturbance(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove all data before the disturbance happens,
    it is said to happen at 1.075s
    """
    where = df['All calculations', 'Time in s'].astype(float) > 1.075
    return df[where]


def angle_transform(df):

    """
    Transform angle data: theta -> (sin(theta), cos(theta))
    creates two new columns and removes the angle column
    note that this adds 10 new columns to the dataframe
    """

    generators = ["G 01", "G 02", "G 03", "G 04", "G 05", "G 06", "G 07", "G 08", "G 09", "G 10"]

    for gen in generators:
        sinangle = np.sin((np.pi/180)*df.loc[:, (gen, "Rotor angle with reference to reference machine angle in deg")])
        cosangle = np.cos((np.pi/180)*df.loc[:, (gen, "Rotor angle with reference to reference machine angle in deg")])
        columns = df.columns
        sinangleind = columns.get_loc((gen, "Rotor angle with reference to reference machine angle in deg"))
        df = df.drop((gen, "Rotor angle with reference to reference machine angle in deg"), axis = 1)
        df.insert(sinangleind, (gen, "sin of rotor angle"), sinangle)
        df.insert(sinangleind+1, (gen, "cos of rotor angle"), cosangle)
    return df


def remove_disturbance(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove all data before the disturbance happens,
    it is said to happen at 1.075s
    """
    where = df['All calculations', 'Time in s'].astype(float) > 1.075
    return df[where]


def remove_pre_cascade_data(df: pd.DataFrame, time_cutoff: float = 0.5) -> pd.DataFrame:
    """
    Remove all data just before cascading occurs. The 'time_cutoff' gives the time before the cascade that we stop the data  
    """
    X = df['All calculations', 'Time in s'].values
    where = df['All calculations', 'Time in s'].astype(float) < X.max() - time_cutoff
    return df[where]


def keep_fixed_time_data(df: pd.DataFrame, time_window: float = 1.3) -> pd.DataFrame:
    """
    Keep only data in a small time_window after the start of the data-set. The 'time_window' gives the time for which we will retain the data  
    """
    X = df['All calculations', 'Time in s'].values
    where = df['All calculations', 'Time in s'].astype(float) < X.min() + time_window
    return df[where]

### Get dictionnary

In [3]:
root_dir = '/data/Data'

file_list_simulations= get_list_simulations(root_dir)
simulation_files_dict = get_dict_scenario_csv(file_list_simulations)

In [4]:
len(simulation_files_dict.keys())

44064

### Gets lists for normalization

In [5]:
means, stds = find_means_and_stds(300, root_dir, simulation_files_dict)

## Launch preprocessing pipeline

In [0]:
os.makedirs(os.path.join('/output/', '01_preprocessed'), exist_ok=True)

In [6]:
load_clean_save_raw_data_by_batch(
    root_dir='/data/Data',
    output_data_dir='/output/01_preprocessed',
    sep=',',
    header=[0,1],
    time_window=10.0,
    means=means,
    stds=stds
)

0 10000_Line 10 - 11_load=0.8_wind=0.4_0.2_0.csv


2 10001_Line 10 - 13_load=0.8_wind=0.4_0.2_0.csv


4 10002_Line 13 - 14_load=0.8_wind=0.4_0.2_0.csv


6 10003_Line 16 - 19_load=0.8_wind=0.4_0.2_0.csv


8 10004_Line 16 - 21_load=0.8_wind=0.4_0.2_0.csv


...

51708 33270_Line 07 - 08_load=1.1_wind=0.6_0.2_0.csv


51710 33271_Line 02 - 25_load=1.1_wind=0.6_0.2_0.csv


51712 33272_Line 01 - 02_load=1.1_wind=0.6_0.2_0.csv


51714 33273_Line 25 - 26_load=1.1_wind=0.6_0.2_0.csv


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 1: invalid continuation byte

In [7]:
df_preprocessed = load(os.path.join('/output/', '01_preprocessed'), '32999_Line 02 - 25_load=1.1_wind=0.4_1.0_0.8.pkl')

In [8]:
df_preprocessed.head()

,All calculations,B_L03,B_L03,B_L04,B_L04,B_L07,B_L07,B_L08,B_L08,B_L12,...,Trf Load_18,Trf Load_20,Trf Load_21,Trf Load_23,Trf Load_24,Trf Load_25,Trf Load_26,Trf Load_27,Trf Load_28,Trf Load_29
0,1.08,-15.256537,-0.534006,-17.536078,1.180491,-17.218269,1.106227,-17.201745,0.985341,-10.282860,...,-1.0,-3.0,-2.0,0.0,2.0,2.0,1.0,-1.0,2.0,2.0
1,1.09,-2.909669,-1.270486,-3.027135,-0.046069,-2.646753,-0.091315,-2.559390,-0.258025,-1.586705,...,-1.0,-3.0,-2.0,0.0,2.0,2.0,1.0,-1.0,2.0,2.0
2,1.10,0.018522,-0.909457,0.226869,0.095115,0.422177,0.016695,0.492962,-0.137941,0.390956,...,-1.0,-3.0,-2.0,0.0,2.0,2.0,1.0,-1.0,2.0,2.0
3,1.11,0.519915,-0.606614,0.578389,0.254168,0.561639,0.153873,0.592281,0.010189,0.765667,...,-1.0,-3.0,-2.0,0.0,2.0,2.0,1.0,-1.0,2.0,2.0
4,1.12,0.662237,-0.471932,0.606397,0.287086,0.452703,0.182846,0.453574,0.043199,0.923509,...,-1.0,-3.0,-2.0,0.0,2.0,2.0,1.0,-1.0,2.0,2.0


In [9]:
df_preprocessed.shape

(999, 260)

In [13]:
df_preprocessed.columns.to_list()

['All calculations',
 'B_L03',
 'B_L03',
 'B_L04',
 'B_L04',
 'B_L07',
 'B_L07',
 'B_L08',
 'B_L08',
 'B_L12',
 'B_L12',
 'B_L15',
 'B_L15',
 'B_L16',
 'B_L16',
 'B_L18',
 'B_L18',
 'B_L20',
 'B_L20',
 'B_L21',
 'B_L21',
 'B_L23',
 'B_L23',
 'B_L24',
 'B_L24',
 'B_L25',
 'B_L25',
 'B_L26',
 'B_L26',
 'B_L27',
 'B_L27',
 'B_L28',
 'B_L28',
 'B_L29',
 'B_L29',
 'Bus 01',
 'Bus 02',
 'Bus 03',
 'Bus 04',
 'Bus 05',
 'Bus 06',
 'Bus 07',
 'Bus 08',
 'Bus 09',
 'Bus 10',
 'Bus 11',
 'Bus 12',
 'Bus 13',
 'Bus 14',
 'Bus 15',
 'Bus 16',
 'Bus 17',
 'Bus 18',
 'Bus 19',
 'Bus 20',
 'Bus 21',
 'Bus 22',
 'Bus 23',
 'Bus 24',
 'Bus 25',
 'Bus 26',
 'Bus 27',
 'Bus 28',
 'Bus 29',
 'Bus 30',
 'Bus 31',
 'Bus 32',
 'Bus 33',
 'Bus 34',
 'Bus 35',
 'Bus 36',
 'Bus 37',
 'Bus 38',
 'Bus 39',
 'Bus_NSG_1',
 'Bus_NSG_2',
 'Bus_NSG_3',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 04

## CAVEAT: the choice was to read preprocessed pickle file without header=[0,1], therefore many columns have the same name ! 

In [14]:
df_preprocessed = load(os.path.join('/output/', '01_preprocessed'), '27889_Line 16 - 24_load=1.0_wind=0.8_0.8_0.8.pkl')

In [15]:
df_preprocessed.head()

,All calculations,B_L03,B_L03,B_L04,B_L04,B_L07,B_L07,B_L08,B_L08,B_L12,...,Trf Load_18,Trf Load_20,Trf Load_21,Trf Load_23,Trf Load_24,Trf Load_25,Trf Load_26,Trf Load_27,Trf Load_28,Trf Load_29
0,1.08,-17.130601,-0.382256,-20.183440,1.127784,-19.905384,1.037016,-19.775711,1.316601,-12.881761,...,-1.0,-3.0,-1.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0
1,1.09,-4.064953,-1.631031,-4.803172,-0.565758,-4.253234,-0.508892,-4.129250,-0.308399,-2.857338,...,-1.0,-3.0,-1.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0
2,1.10,0.808676,-1.439031,0.747398,-0.532650,0.890798,-0.475366,0.943446,-0.268303,0.874277,...,-1.0,-3.0,-1.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0
3,1.11,1.262567,-1.162791,1.159679,-0.369235,1.081325,-0.333735,1.093768,-0.116468,1.355582,...,-1.0,-3.0,-1.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0
4,1.12,1.456352,-1.054539,1.302460,-0.339407,1.052173,-0.307640,1.026694,-0.088175,1.657950,...,-1.0,-3.0,-1.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0


In [17]:
df_preprocessed.shape

(999, 260)

In [18]:
df_preprocessed.columns.to_list()

['All calculations',
 'B_L03',
 'B_L03',
 'B_L04',
 'B_L04',
 'B_L07',
 'B_L07',
 'B_L08',
 'B_L08',
 'B_L12',
 'B_L12',
 'B_L15',
 'B_L15',
 'B_L16',
 'B_L16',
 'B_L18',
 'B_L18',
 'B_L20',
 'B_L20',
 'B_L21',
 'B_L21',
 'B_L23',
 'B_L23',
 'B_L24',
 'B_L24',
 'B_L25',
 'B_L25',
 'B_L26',
 'B_L26',
 'B_L27',
 'B_L27',
 'B_L28',
 'B_L28',
 'B_L29',
 'B_L29',
 'Bus 01',
 'Bus 02',
 'Bus 03',
 'Bus 04',
 'Bus 05',
 'Bus 06',
 'Bus 07',
 'Bus 08',
 'Bus 09',
 'Bus 10',
 'Bus 11',
 'Bus 12',
 'Bus 13',
 'Bus 14',
 'Bus 15',
 'Bus 16',
 'Bus 17',
 'Bus 18',
 'Bus 19',
 'Bus 20',
 'Bus 21',
 'Bus 22',
 'Bus 23',
 'Bus 24',
 'Bus 25',
 'Bus 26',
 'Bus 27',
 'Bus 28',
 'Bus 29',
 'Bus 30',
 'Bus 31',
 'Bus 32',
 'Bus 33',
 'Bus 34',
 'Bus 35',
 'Bus 36',
 'Bus 37',
 'Bus 38',
 'Bus 39',
 'Bus_NSG_1',
 'Bus_NSG_2',
 'Bus_NSG_3',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 04

In [19]:
df_preprocessed = load(os.path.join('/output/', '01_preprocessed'), '10000_Line 10 - 11_load=0.8_wind=0.4_0.2_0.pkl')

In [20]:
df_preprocessed.head()

,All calculations,B_L03,B_L03,B_L04,B_L04,B_L07,B_L07,B_L08,B_L08,B_L12,...,Trf Load_18,Trf Load_20,Trf Load_21,Trf Load_23,Trf Load_24,Trf Load_25,Trf Load_26,Trf Load_27,Trf Load_28,Trf Load_29
0,1.08,-14.831917,-3.116153,-21.445156,-2.145748,-16.815255,-2.019509,-14.594221,-2.188274,-23.408232,...,1.0,-2.0,1.0,2.0,4.0,4.0,3.0,2.0,3.0,3.0
1,1.09,-0.954320,-2.338654,-2.203433,-1.544177,-1.830124,-1.332780,-1.462105,-1.468340,-2.947133,...,1.0,-2.0,1.0,2.0,4.0,4.0,3.0,2.0,3.0,3.0
2,1.10,0.609989,-1.749531,0.076816,-1.097977,0.123308,-0.888890,0.263617,-1.002657,-0.078107,...,1.0,-2.0,1.0,2.0,4.0,4.0,3.0,2.0,3.0,3.0
3,1.11,1.322034,-1.330827,1.095565,-0.793194,0.908792,-0.617845,0.937556,-0.718008,1.322354,...,1.0,-2.0,1.0,2.0,4.0,4.0,3.0,2.0,3.0,3.0
4,1.12,1.822901,-1.085222,1.774333,-0.642590,1.356201,-0.496763,1.306071,-0.590432,2.332859,...,1.0,-2.0,1.0,2.0,4.0,4.0,3.0,2.0,3.0,3.0


In [21]:
df_preprocessed.shape

(996, 260)

In [22]:
df_preprocessed.columns.to_list()

['All calculations',
 'B_L03',
 'B_L03',
 'B_L04',
 'B_L04',
 'B_L07',
 'B_L07',
 'B_L08',
 'B_L08',
 'B_L12',
 'B_L12',
 'B_L15',
 'B_L15',
 'B_L16',
 'B_L16',
 'B_L18',
 'B_L18',
 'B_L20',
 'B_L20',
 'B_L21',
 'B_L21',
 'B_L23',
 'B_L23',
 'B_L24',
 'B_L24',
 'B_L25',
 'B_L25',
 'B_L26',
 'B_L26',
 'B_L27',
 'B_L27',
 'B_L28',
 'B_L28',
 'B_L29',
 'B_L29',
 'Bus 01',
 'Bus 02',
 'Bus 03',
 'Bus 04',
 'Bus 05',
 'Bus 06',
 'Bus 07',
 'Bus 08',
 'Bus 09',
 'Bus 10',
 'Bus 11',
 'Bus 12',
 'Bus 13',
 'Bus 14',
 'Bus 15',
 'Bus 16',
 'Bus 17',
 'Bus 18',
 'Bus 19',
 'Bus 20',
 'Bus 21',
 'Bus 22',
 'Bus 23',
 'Bus 24',
 'Bus 25',
 'Bus 26',
 'Bus 27',
 'Bus 28',
 'Bus 29',
 'Bus 30',
 'Bus 31',
 'Bus 32',
 'Bus 33',
 'Bus 34',
 'Bus 35',
 'Bus 36',
 'Bus 37',
 'Bus 38',
 'Bus 39',
 'Bus_NSG_1',
 'Bus_NSG_2',
 'Bus_NSG_3',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 01',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 02',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 03',
 'G 04